In [ ]:
import baltic as bt
import requests
from io import StringIO as sio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Polygon, PathPatch
from matplotlib.path import Path
from matplotlib.collections import LineCollection
from matplotlib import cm
import matplotlib.patheffects as path_effects
import numpy as np
from datetime import datetime as dt
import os, re
import PyAstronomy
from PyAstronomy import pyasl
import datetime
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Polygon
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
mpl.rcParams['font.family'] = 'sans-serif'
from scipy import stats
import pandas as pd

def get_all_terminals(node):
    terminals = []
    for i in node.children:
        if i.branchType == "leaf":
            terminals.append(i)
        if i.branchType != "leaf":
            terminals.extend(get_all_terminals(i))
    return terminals

treeFile="hMPXV.State.history_MCC.nwk" ## stream from repo copy

ll=bt.loadNexus(treeFile) ## treeFile here can alternatively be a path to a local file
ll.traverse_tree()
ll.treeStats()
ll.sortBranches()

for i in ll.Objects:
    if i.branchType == "leaf":
        if "VSP" in i.name:
            i.traits["rate_signDistribution"]="NEW"

        elif "TRM" in i.name:
            i.traits["rate_signDistribution"]="NEW"
        else:
            i.traits["rate_signDistribution"]="NO"


In [ ]:
alles=[]
for i in ll.Objects:
    alles.append(i.traits["State.states"])
alles=list(set(alles))

df=pd.read_csv("State_class.txt", sep="\t")
mmap=dict(zip(df["taxa"], df["class"]))
mmap.update({'Akwa-Ibom':"S"})
my_dict = {key: [] for key in alles}
print(my_dict)

In [ ]:
## Borrowed from Gytis Dudas largely

tree_strings={'ogun': [], 'lagos': [], 'enugu': [], 'uk': [], 'bayelsa': [], 
'benue': [], 'israel': [], 'portugal': [], 'ondo': [], 'fct': [], 'delta': [], 
'edo': [], 'kogi': [], 'austria': [], 'Akwa-Ibom': [], 'abia': [], 'niger': [], 
'nigeria': [], 'ebonyi': [], 'india': [], 'plateau': [], 'borno': [], 'slovenia': [], 
'kwara': [], 'usa': [], 'singapore': [], 'egypt': [], 'nasarawa': [], 'Cross-River': [], 
'rivers': [], 'gombe': [], 'kano': [], 'anambra': [], 'kaduna': [], 'imo': [], 'thailand': [], 'oyo': [],'ancestor':[],
             'cameroon': []}

subtype_trees={'ogun': [], 'lagos': [], 'enugu': [], 'uk': [], 'bayelsa': [], 
'benue': [], 'israel': [], 'portugal': [], 'ondo': [], 'fct': [], 'delta': [], 
'edo': [], 'kogi': [], 'austria': [], 'Akwa-Ibom': [], 'abia': [], 'niger': [], 
'nigeria': [], 'ebonyi': [], 'india': [], 'plateau': [], 'borno': [], 'slovenia': [], 
'kwara': [], 'usa': [], 'singapore': [], 'egypt': [], 'nasarawa': [], 'Cross-River': [], 
'rivers': [], 'gombe': [], 'kano': [], 'anambra': [], 'kaduna': [], 'imo': [], 'thailand': [], 'oyo': [],'ancestor':[],
              'cameroon': []}
traitName='State.states'
branchWidth=2
tipSize=30

for l in ll.Objects:
    k=l
    kp=l.parent
    kloc=k.traits[traitName]
    if kloc != "cameroon":
        if traitName in k.parent.traits:
            kploc=kp.traits[traitName]
            kpc=kploc
        else:
            kploc='ancestor'
            kpc='ancestor'
        kc=kloc
        if kc!=kpc:
            traverse_condition=lambda w:w.traits[traitName]==kc
            subtree=ll.subtree(k,traverse_condition=traverse_condition) 
            if subtree: 
                subtree.traverse_tree()
                subtree.sortBranches()
                tree_strings[kc].append(subtree.toString()) 
                subtype_trees[kc].append((kpc,subtree))

keys_to_delete=[xx for xx in list(mmap.keys()) if xx not in list(subtype_trees.keys())]

for key in keys_to_delete:
    mmap.pop(key, None) 

mmap.pop('rivers', None) 
mmap.update({'rivers': 'S'})
original_dict = subtype_trees
grouping_dict = mmap
sorted_dict = {}
for key, group in sorted(grouping_dict.items(), key=lambda x: x[1]):  # Sort by grouping
    if key!= "cameroon":
        sorted_dict.setdefault(group, {})[key] = original_dict[key]

order_list=['NC', 'NE', 'NW', 'Rest', 'SE', 'SW', 'S', 'nigeria']
sorted_dict_2 = {key: sorted_dict[key] for key in order_list if key in sorted_dict}
non_nigeria=['uk', 'israel', 'portugal', 'austria', 'india', 'slovenia', 'usa', 'singapore', 'egypt', 'thailand','Cameroon']

gg=[]
for region in list(sorted_dict_2.keys()):
    for subtype in sorted_dict_2[region].keys():
        if subtype not in ["nigeria", "Rest"]:
            if subtype not in non_nigeria:
                gg.append(subtype)
RN_dd = dict(zip(gg, [s.capitalize() for s in gg]))
RN_d={}
for k, v in RN_dd.items():
    v=re.sub("ibom", "Ibom",re.sub("Crossriver", "Cross River",re.sub("Fct", "FCT", v)))
    RN_d[k]=v

In [ ]:
fig,ax = plt.subplots(figsize=(10,14),facecolor='w')
traitName='State.states'
tipSize=20
cumulative_y=0
x_attr=lambda k: k.absoluteTime

dd={  'Rest': "#C0C0C0",
   "nigeria": "black",
    'NC': "#960018",
  'NE': "#FE5A1D",
  'NW': "#FFD700",
  'SE': "#0070BB",
  'SW': "#138808",
    'S': "#16166B",
   "Cameroon": "grey"}

def c_func(k):
    if k.traits["State.states"] != "rivers":
        region=mmap[k.traits["State.states"]]
        p=dd[region]
    else:
        p="#273BE2"
    return(p)


for region in list(sorted_dict_2.keys()):
    for subtype in sorted_dict_2[region].keys():
        if subtype not in ["nigeria", "Rest", "Cameroon"]:
            place_y=[]
            if subtype not in non_nigeria:
                if subtype !=  "Cameroon":
                    gg.append(subtype)
                    for t,tr in enumerate(sorted_dict_2[region][subtype]): 
                        origin,loc_tree=tr ## get origin of subtree, subtree itself
                        if origin != "cameroon" or subtype == "rivers":
                            y_attr=lambda k: k.y+cumulative_y
                            loc_tree.plotTree(ax,x_attr=x_attr,y_attr=y_attr,colour=c_func)
                            loc_tree.plotPoints(ax,x_attr=x_attr,y_attr=y_attr,size=20,colour=c_func,zorder=100)
                            oriC='dimgrey' if origin=='ancestor' else c_func(loc_tree.root.parent)
                            oriX=loc_tree.root.absoluteTime-loc_tree.root.length
                            oriY=loc_tree.root.y+cumulative_y
                            ax.scatter(oriX,oriY,50,facecolor=oriC,edgecolor='black',lw=1,zorder=100) ## add big circle at base of tree to indicate origin
                            place_y.append(cumulative_y)
                            cumulative_y+=loc_tree.ySpan+5 ## increment y displacement

                    if region == "S" and subtype == "rivers":
                        clade_xx2=plt.Polygon(([2015, place_y[0]],
                                           [2015,700],
                                           [2025,700],
                                           [2025,place_y[0]]),
                                          facecolor=None,
                                          edgecolor=dd[region],zorder=0,
                                          alpha=0.1)
                        ax.add_patch(clade_xx2)
                        ax.text(2023.5, 635, RN_d[subtype], size=10)
                    else:
                        clade_xx=plt.Polygon(([2015, place_y[0]],
                                           [2015,place_y[-1]+2],
                                           [2025,place_y[-1]+2],
                                           [2025,place_y[0]]),
                                          facecolor=None,
                                          edgecolor=dd[region],zorder=0,
                                          alpha=0.05)
                        ax.add_patch(clade_xx) 
                        ax.text(2023.5, (place_y[0]+place_y[-1])/2, RN_d[subtype], size=10)

clade=plt.Polygon(([2015, cumulative_y],
                   [2015,415],
                   [2025,415],
                   [2025,cumulative_y]),
                  facecolor="#16166B",
                  edgecolor='black',zorder=1,
                  alpha=0.15)
ax.add_patch(clade)        

clade2=plt.Polygon(([2015, 415],
                   [2015,311],
                   [2025,311],
                   [2025,415]),
                  facecolor="#138808",
                  edgecolor='black',zorder=1,
                  alpha=0.15)
ax.add_patch(clade2)  

clade3=plt.Polygon(([2015,310 ],
                   [2015,128],
                   [2025,128],
                   [2025,310]),
                  facecolor="#0070BB",
                  edgecolor='black',zorder=1,
                  alpha=0.15)
ax.add_patch(clade3)

clade4=plt.Polygon(([2015,128 ],
                   [2015,99],
                   [2025,99],
                   [2025,128]),
                  facecolor="#FFD700",
                  edgecolor='black',zorder=1,
                  alpha=0.15)
ax.add_patch(clade4)  

clade5=plt.Polygon(([2015,99 ],
                   [2015,80],
                   [2025,80],
                   [2025,95]),
                  facecolor="#FE5A1D",
                  edgecolor='black',zorder=1,
                  alpha=0.15)
ax.add_patch(clade5)  


clade5=plt.Polygon(([2015,80 ],
                   [2015,-10],
                   [2025,-10],
                   [2025,80]),
                  facecolor="#960018",
                  edgecolor='black',zorder=1,
                  alpha=0.15)
ax.add_patch(clade5)  
            
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()
[ax.spines[loc].set_visible(False) for loc in ['top','right','left']]
ax.tick_params(axis='y',size=0)
ax.set_yticklabels([])
ax.set_ylim(-10,cumulative_y)
ax.set_xlim(2015,2025)
plt.xticks(range(2015, 2025, 1), fontsize=15) 

plt.axvline(x=2017.71, color='r', linestyle='--')  

plt.savefig("Transmission_lineage.png",bbox_inches='tight', dpi=1000)   


In [ ]:
MT=[]

for region in list(sorted_dict_2.keys()):
    for subtype in sorted_dict_2[region].keys():
        if subtype not in ["nigeria", "Rest"]:
            if subtype not in non_nigeria:
                if subtype !=  "Cameroon":
                    for t,tr in enumerate(sorted_dict_2[region][subtype]): 
                        origin,loc_tree=tr ## get origin of subtree, subtree itself
                        if origin != "cameroon" or subtype == "rivers":
                            heights=[]
                            if subtype =="rivers" :
                                print("rivers",loc_tree.root.absoluteTime)
                            for k in loc_tree.Objects:
                                heights.append(k.height)
                            ori=loc_tree.root.absoluteTime-loc_tree.root.length
                            df=pd.DataFrame([region, subtype, np.max(heights)*365, 
                                             ori]).T
                            MT.append(df)

MTdf = pd.concat(MT)

In [ ]:
dd={  'Rest': "#C0C0C0",
   "nigeria": "black",
    'NC': "#960018",
  'NE': "#FE5A1D",
  'NW': "#FFD700",
  'SE': "#0070BB",
  'SW': "#138808",
    'S': "#16166B",
   "Cameroon": "grey"}


MTdf=MTdf.rename(columns={0:"Region", 1:"State", 2:"persistence", 3:"Start"})
df=pd.DataFrame(MTdf.groupby("State")["persistence"].median()).reset_index()
sta_reg=dict(zip(MTdf["State"], MTdf["Region"]))
df=df.sort_values("persistence", ascending=False)
df["Region"]=df["State"].map(sta_reg)
df["year"]=df["persistence"]/365

MTdf["col"]=MTdf["Region"].map(dd)

In [ ]:
fig, ax = plt.subplots( figsize=(3.5,4) )

plt.yticks(range(0, 2905+365, 365), range(0, 9,1 ))
plt.grid(axis='y', color='lightgrey', linestyle='--', linewidth=1)
plt.scatter(MTdf["Start"], MTdf["persistence"], c=MTdf["col"], 
            edgecolor='black', s=100, zorder=6000)

plt.ylabel('Persistence in years', fontsize=15)
plt.xlabel('Origin', fontsize=15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=13)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


#plt.show()

plt.savefig("Persistence.png",bbox_inches='tight', dpi=1000)   


In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def months_between_dates(start_date, end_date):
    # Convert string inputs to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    months = []
    current_month = start_date
    while current_month < end_date:
        months.append(current_month.strftime('%Y-%m'))
        current_month += relativedelta(months=1)
    return months
    
MT=[]

for region in list(sorted_dict_2.keys()):
    for subtype in sorted_dict_2[region].keys():
        if subtype not in ["nigeria", "Rest", "Cameroon"]:
            if subtype not in non_nigeria:
                for t,tr in enumerate(sorted_dict_2[region][subtype]): 
                    origin,loc_tree=tr ## get origin of subtree, subtree itself
                    if origin != "cameroon" or subtype == "rivers":
                        heights=[]
                        if subtype =="rivers" :
                            print("rivers",loc_tree.root.absoluteTime)
                        for k in loc_tree.Objects:
                            heights.append(k.height)
                        ori=loc_tree.root.absoluteTime-loc_tree.root.length
                        start_date=(pyasl.decimalYearGregorianDate(ori, "yyyy-mm-dd"))
                        end_date=(pyasl.decimalYearGregorianDate(ori+np.max(heights), "yyyy-mm-dd"))
                        months = months_between_dates(start_date, end_date)
                        df=pd.DataFrame([region, subtype, start_date, end_date, months]).T
                        MT.append(df)
MTdf = pd.concat(MT)

In [ ]:
start_date="2015-03-01"
end_date="2023-05-31"
months = months_between_dates(start_date, end_date)

In [ ]:
alles={}
for mon in months:
    circ_state=[]

    for r in MTdf.iterrows():
        # print(r[1][1],r[1][4])
        if mon in r[1][4]:
            # print(r[1][1])
            circ_state.append(r[1][1])
    alles[mon]=circ_state
        

In [ ]:
alles_dfl=[]

for k,v in alles.items():
    sdf=pd.DataFrame(pd.Series(v).value_counts()).reset_index()
    sdf["month"]=k
    alles_dfl.append(sdf)
    
adf = pd.concat(alles_dfl)
adf["Region"]=adf["index"].map(mmap)

grouped_df=adf.groupby(["month", "Region"])["count"].agg(sum).reset_index()


In [ ]:
data_l=[]

for reg in grouped_df["Region"].unique():
    ddf=grouped_df[grouped_df["Region"]==reg]
    have=ddf["month"].to_list()
    missing=[xx for xx in months if xx not in have]
    sdl=[]
    sdl.append(ddf)
    for m in missing:
        new_row = pd.DataFrame(pd.Series([m, reg, 0], index=ddf.columns)).T
        sdl.append(new_row)
    fulldf = pd.concat(sdl)
    fulldf=fulldf.sort_values("month")
    data_l.append(fulldf)
fdf = pd.concat(data_l)

group_data=[]
for reg in fdf["Region"].unique():
    group_data.append(fdf[fdf["Region"]==reg]["count"])


categories = list(fdf["month"].unique())
regions=list(fdf["Region"].unique())
plt.figure(figsize=(8, 6))

group_data = np.array(group_data)

# fig, ax = plt.subplots( dpi=200, figsize=(3.5,4) )

plt.figure(figsize=(3.5, 4))

bottom = [0]*len(group_data[0])

for i in range(len(group_data)):
    plt.bar(categories, group_data[i], bottom=bottom, label=re.sub("S$", "SS", regions[i]), 
            color=dd[regions[i]])
    bottom += group_data[i]  


plt.xticks(range(0, len(categories), 24), categories[::24])
plt.legend(frameon=False)
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.ylabel('Number of transmission chains circulating', fontsize=10)
plt.axvline(x="2017-09", color='r', linestyle='--')  # Plot horizontal line at y=3
plt.tick_params(axis='x', labelsize=8)

#plt.show()
plt.savefig("Lineages_circulating_nr.png",bbox_inches='tight', dpi=1000)   

